In [1]:
# the subbort that had a major move will be considerred more reliable
# same for resistance

In [2]:
import pandas as pd
import numpy as np
# import yfinance

# from mplfinance import candlestick_ohlc
from mplfinance.original_flavor import candlestick_ohlc

import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

import plotly.graph_objects as go

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
# plt.rcParams['figure.figsize'] = [12, 7]

# plt.rc('font', size=14) 

In [4]:

name = 'BTC-USD'
# ticker = yfinance.Ticker(name)
# df = ticker.history(interval="90m")

# # df = ticker.history(interval="1d",start="2020-03-15",end="2020-07-15")

df=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\4h\ZILUSDT_4h.csv",index_col=['Time'], parse_dates=['Time'])


# df = df.reset_index().set_index('Time') # SWITCH INDEX AND TIME COLumn places

# df = df.set_index('Time')

# df['Date'] = df['Time']
# pd.to_datetime(df['Time'])
# pd.to_datetime(df['Date'])
# df = df.set_index('Time')


In [5]:

df

,Open,High,Low,Close,Volume
Time,,,,,
2021-08-23 00:00:00,0.10950,0.11300,0.10820,0.11257,44031225.7
2021-08-23 04:00:00,0.11255,0.11500,0.11185,0.11369,53335601.5
2021-08-23 08:00:00,0.11372,0.11624,0.11241,0.11546,43975667.2
2021-08-23 12:00:00,0.11546,0.12231,0.11443,0.11662,183383243.5
2021-08-23 16:00:00,0.11670,0.11775,0.11252,0.11313,56409604.1
...,...,...,...,...,...
2022-02-04 16:00:00,0.04892,0.04942,0.04862,0.04922,35741974.4
2022-02-04 20:00:00,0.04924,0.05015,0.04894,0.04989,25040627.2
2022-02-05 00:00:00,0.04988,0.05069,0.04906,0.05059,48485326.8


In [6]:
df['Date'] = pd.to_datetime(df.index)
# change datetime into matplotlib time
# does not work with plotly
# df['Date'] = df['Date'].apply(mpl_dates.date2num)

df = df.loc[:,['Date' ,'Open', 'High', 'Low', 'Close']]

In [7]:
df


,Date,Open,High,Low,Close
Time,,,,,
2021-08-23 00:00:00,2021-08-23 00:00:00,0.10950,0.11300,0.10820,0.11257
2021-08-23 04:00:00,2021-08-23 04:00:00,0.11255,0.11500,0.11185,0.11369
2021-08-23 08:00:00,2021-08-23 08:00:00,0.11372,0.11624,0.11241,0.11546
2021-08-23 12:00:00,2021-08-23 12:00:00,0.11546,0.12231,0.11443,0.11662
2021-08-23 16:00:00,2021-08-23 16:00:00,0.11670,0.11775,0.11252,0.11313
...,...,...,...,...,...
2022-02-04 16:00:00,2022-02-04 16:00:00,0.04892,0.04942,0.04862,0.04922
2022-02-04 20:00:00,2022-02-04 20:00:00,0.04924,0.05015,0.04894,0.04989
2022-02-05 00:00:00,2022-02-05 00:00:00,0.04988,0.05069,0.04906,0.05059


In [8]:
def isSupport(df,i):
  # fractal (i-2)>(i-1)>i<(i+1)<(i+2)
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

  return resistance

In [9]:
levels = []
# start from 2 because i need i-2
# end at shape[0]-2 because  i need i+2
# i is the number of current row
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    levels.append((i,df['Low'][i]))
  elif isResistance(df,i):
    levels.append((i,df['High'][i]))

In [10]:
# MOCHLV = zip(df.Date, df.Open, df.Close, df.High, df.Low, df.Volume)


In [11]:
# def plot_all():
#   fig, ax = plt.subplots()

#   candlestick_ohlc(ax,df.values,width=0.6, \
#                    colorup='green', colordown='red', alpha=0.8)
#   # candlestick_ohlc(ax,MOCHLV)

#   date_format = mpl_dates.DateFormatter('%d %b %Y')
#   ax.xaxis.set_major_formatter(date_format)
#   fig.autofmt_xdate()

#   fig.tight_layout()

#   for level in levels:
#     plt.hlines(level[1],xmin=df['Date'][level[0]],\
#                xmax=max(df['Date']),colors='blue')
#   fig.show()

In [12]:
def plot_all():

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    for level in levels:
        # fig.add_hline(level[1])
        fig.add_shape(type='line',
                x0=df['Date'][level[0]],
                y0=level[1],
                x1=max(df['Date']),
                y1=level[1],
                line=dict(color='black',),
                xref='x',
                yref='y')
    
    fig.layout.xaxis.rangeslider.visible = False


    fig.show()



#   fig, ax = plt.subplots()

#   candlestick_ohlc(ax,df.values,width=0.6, \
#                    colorup='green', colordown='red', alpha=0.8)
#   # candlestick_ohlc(ax,MOCHLV)

#   date_format = mpl_dates.DateFormatter('%d %b %Y')
#   ax.xaxis.set_major_formatter(date_format)
#   fig.autofmt_xdate()

#   fig.tight_layout()

#   for level in levels:
#     plt.hlines(level[1],xmin=df['Date'][level[0]],\
#                xmax=max(df['Date']),colors='blue')
#   fig.show()



In [13]:
# plot_all()

In [14]:
s =  np.mean(df['High'] - df['Low'])

In [15]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [16]:
levels = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    l = df['Low'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

  elif isResistance(df,i):
    l = df['High'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

In [17]:
levels

[(3, 0.12231),
 (5, 0.11224),
 (10, 0.10471),
 (18, 0.1185),
 (74, 0.12588),
 (89, 0.13374),
 (93, 0.08562),
 (98, 0.09161),
 (129, 0.09623),
 (174, 0.08118),
 (205, 0.0765),
 (272, 0.1),
 (619, 0.05264),
 (632, 0.05995),
 (668, 0.06311),
 (751, 0.07074),
 (821, 0.06692),
 (843, 0.0561),
 (920, 0.04511),
 (927, 0.03758),
 (940, 0.04847)]

In [18]:
plot_all()